Cryptocurrency trading bot.

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

# File:        cryptocurrency/crypto_logger_base.py
# By:          Samuel Duclos
# For          Myself
# Description: Simple Binance logger base class.

# Library imports.
from cryptocurrency.resample import resample
from binance.client import Client
from abc import abstractmethod, ABC
from time import sleep, time
from os.path import exists, join
from os import mkdir

import pandas as pd

class Crypto_logger_base(ABC):
    def __init__(self, interval='15s', delay=4.7, buffer_size=3000, directory='crypto_logs', 
                 log_name='crypto_log', raw=False):
        """
        :param interval: OHLCV interval to log. Default is 15 seconds.
        :param delay: delay between Binance API requests. Minimum calculated was 4.7 seconds.
        :param buffer_size: buffer size to avoid crashing on memory accesses.
        :param directory: the directory where to output the logs.
        :param log_name: name of the log file.
        :param raw: whether the log dumps raw (instantaneous) or OHLCV data.
        """
        self.interval = interval
        self.delay = delay
        self.buffer_size = buffer_size
        self.directory = directory
        self.raw = raw

        self.log_name = join(self.directory, log_name + '.txt')
        self.log_screened_name = join(self.directory, log_name + '_screened.txt')

        if not exists(self.directory):
            mkdir(self.directory)

    #self.get_from_file(log_name=self.log_name, from_raw=False)
    #self.get_from_file(log_name=self.input_log_name, from_raw=self.load_from_ohlcv)
    def get_from_file(self, log_name, from_raw=False):
        if from_raw:
            dataset = pd.read_csv(log_name, header=0, index_col=0)
        else:
            dataset = pd.read_csv(log_name, header=[0, 1], index_col=0)
        dataset.index = pd.DatetimeIndex(dataset.index)
        return dataset.sort_index(axis='index')

    @abstractmethod
    def get(self, **kwargs):
        raise NotImplementedError()

    @abstractmethod
    def screen(self, **kwargs):
        raise NotImplementedError()

    def put(self, dataset):
        dataset = dataset.copy().reset_index()
        if self.raw:
            dataset = dataset.drop_duplicates(subset=['symbol', 'count'], 
                                              keep='first', ignore_index=True)
        else:
            dataset = dataset.drop_duplicates(keep='last', ignore_index=True)

        if 'date' in dataset.columns:
            min_index_int = dataset[dataset['date'] == self.min_index].index[0]
            dataset = dataset.set_index('date')
        if not self.raw:
            dataset = resample(dataset, self.interval)
        if 'date' in dataset.columns:
            dataset = dataset.iloc[min_index_int:]

        dataset = dataset.tail(self.buffer_size)
        dataset.to_csv(self.log_name)
        self.min_index = dataset.index[0]
        return dataset

    def start(self, append=False, roll=0):
        """Main logger loop."""
        print('Starting crypto logger.')

        if exists(self.log_name) and 'output' in self.log_name:
            self.dataset = self.get_from_file(log_name=self.log_name, from_raw=False)
            self.dataset = self.dataset.tail(self.buffer_size)
        else:
            self.dataset = self.get()

        self.min_index = self.dataset.index[-1]
        self.dataset = self.put(self.dataset)

        while True:
            try:
                dataset = pd.concat([self.dataset, self.get()], axis='index', join='outer')
            except (KeyboardInterrupt, SystemExit):
                print('User terminated crypto logger process.')
                break
            except Exception as e:
                print(e)
            try:
                self.dataset = self.put(dataset)
            except (KeyboardInterrupt, SystemExit):
                print('Saving latest complete dataset...')
                self.dataset = self.put(dataset)
                print('User terminated crypto logger process.')
                break
            except Exception as e:
                print(e)
            try:
                if exists(self.log_screened_name):
                    dataset_screened_old = \
                        pd.read_csv(self.log_screened_name, index_col=0, header=0)
                else:
                    dataset_screened_old = None
                dataset_screened = self.screen(self.dataset)
                if dataset_screened is not None:
                    if roll != 0:
                        if append and exists(self.log_screened_name):
                            dataset_screened = \
                                pd.concat([dataset_screened_old, dataset_screened], axis='index')
                            dataset_screened = \
                                dataset_screened.drop_duplicates(subset=['symbol'], keep='last')
                        dataset_screened = dataset_screened.tail(roll)
                        dataset_screened.to_csv(self.log_screened_name)
                    elif append:
                        dataset_screened.to_csv(self.log_screened_name, mode='a')
                    else:
                        dataset_screened.to_csv(self.log_screened_name)
            except (KeyboardInterrupt, SystemExit):
                print('User terminated crypto logger process.')
                break
            except Exception as e:
                print(e)
            sleep(self.delay)
        print('Crypto logger process done.')

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

# File:        cryptocurrency/crypto_logger_input.py
# By:          Samuel Duclos
# For          Myself
# Description: Simple Binance logger circular buffered for N time precision.

# Library imports.
#from cryptocurrency.crypto_logger_base import Crypto_logger_base
from cryptocurrency.authentication import Cryptocurrency_authenticator
from cryptocurrency.exchange import Cryptocurrency_exchange
from cryptocurrency.conversion_table import get_conversion_table, get_tradable_tickers_info
from os import mkdir
from os.path import exists, join

import datetime
import pandas as pd
pd.options.mode.chained_assignment = None

class Crypto_logger_input(Crypto_logger_base):
    def __init__(self, delay=4.7, interval='15s', buffer_size=3000, 
                 price_percent=5.0, volume_percent=0.0, as_pair=False):
        """
        :param interval: OHLCV interval to log. Default is 15 seconds.
        :param delay: delay between Binance API requests. Minimum calculated was 4.7 seconds.
        :param buffer_size: buffer size to avoid crashing on memory accesses.
        :param price_percent: price move percent.
        :param volume_percent: volume move percent.
        """
        self.resample = None
        self.price_percent = price_percent
        self.volume_percent = volume_percent
        self.as_pair = as_pair
        super().__init__(interval=interval, delay=delay, buffer_size=buffer_size, 
                         directory='crypto_logs', log_name='crypto_input_log_' + interval, 
                         raw=True)

        authenticator = Cryptocurrency_authenticator(use_keys=False, testnet=False)
        self.client = authenticator.spot_client

        exchange = Cryptocurrency_exchange(client=self.client, directory=self.directory)
        self.exchange_info = exchange.info

    def filter_movers(self, dataset, count=1000, price_percent=5.0, volume_percent=0.0):
        dataset = dataset.reset_index()
        dataset[['price_change_percent', 'rolling_quote_volume']] = \
            dataset[['price_change_percent', 'rolling_quote_volume']].astype(float)
        dataset['last_price_move'] = dataset['price_change_percent'].copy()
        dataset['last_volume_move'] = dataset['rolling_quote_volume'].copy()
        movers = dataset.groupby(['symbol'])
        dataset = dataset.drop(columns=['last_price_move', 'last_volume_move'])
        price_movers = movers['last_price_move']
        volume_movers = movers['last_volume_move']
        price_movers = price_movers.agg(lambda x: x.diff(1).abs().iloc[-1])
        volume_movers = volume_movers.agg(lambda x: (100 * x.pct_change(1)).iloc[-1])
        price_movers = price_movers.sort_values(ascending=False)
        volume_movers = volume_movers.sort_values(ascending=False)
        price_movers = price_movers[price_movers > 0.0]
        price_movers = price_movers.to_frame(name='last_price_move')
        volume_movers = volume_movers.to_frame(name='last_volume_move')
        movers = pd.concat([price_movers, volume_movers], axis='columns')
        movers = movers.reset_index()
        price_movers_mask = movers['last_price_move'] > price_percent
        volume_movers_mask = movers['last_volume_move'] > volume_percent
        movers = movers[price_movers_mask & volume_movers_mask]
        movers = movers.sort_values(by=['last_volume_move', 'last_price_move'], ascending=False)
        movers = movers.tail(count).reset_index(drop=True)
        return dataset.merge(right=movers, how='right', on=['symbol']).set_index('date')

    def screen(self, dataset):
        dataset = get_tradable_tickers_info(dataset, as_pair=self.as_pair)
        dataset = self.filter_movers(dataset, count=1000, 
                                     price_percent=self.price_percent, 
                                     volume_percent=self.volume_percent)
        return dataset.drop_duplicates(subset=['symbol', 'count'], keep='last')

    def prepare_downsampling(self, dataset):
        dataset['close_time'] /= 1000
        dataset['close_time'] = \
            dataset['close_time'].apply(datetime.datetime.fromtimestamp)
        dataset['date'] = pd.DatetimeIndex(dataset['close_time']).round(self.interval)
        return dataset.set_index('date').sort_index()

    def get(self):
        """Get all pairs data from Binance API."""
        dataset = get_conversion_table(self.client, self.exchange_info)
        self.conversion_table = dataset.copy()
        self.conversion_table.to_csv(self.log_name.replace('.txt', '') + '_conversion_table.txt')
        return self.prepare_downsampling(self.conversion_table)

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

# File:        crypto_logger_input_1min.py
# By:          Samuel Duclos
# For          Myself
# Description: Simple Binance logger output for the 1 minute interval.

# Library imports.
#from cryptocurrency.crypto_logger_input import Crypto_logger_input

crypto_logger_input_1min = Crypto_logger_input(delay=12, interval='1min', buffer_size=20000, 
                                               price_percent=1.0, volume_percent=1.0)
crypto_logger_input_1min.start(append=True, roll=60)

In [ ]:
import pandas as pd

crypto_input_log_1min = 'crypto_logs/crypto_input_log_1min.txt'
df = pd.read_csv(crypto_input_log_1min, header=0, index_col=0)
df.index = pd.to_datetime(df.index, utc=True)
df

In [1]:
import pandas as pd

crypto_input_log_15s = 'crypto_logs/crypto_input_log_15s.txt'
df = pd.read_csv(crypto_input_log_15s, header=0, index_col=0)
df.index = pd.to_datetime(df.index)
df

,base_asset,price_change_percent,close_time,last_ID,count,rolling_base_volume,bid_volume,ask_volume,close,bid_price,ask_price,bid_ask_change_percent,bid_ask_volume_percent,rolling_quote_volume,symbol,quote_asset
date,,,,,,,,,,,,,,,,
2022-11-04 21:12:45,BDOT,111.412412,2022-11-04 21:12:42.700,2.330540e+05,612.0,1.656074e+05,2.846446e+00,2.510965e+02,5.349477,3.947914e+00,3.958078e+00,0.256791,1.120900,1.656074e+05,BDOT,BDOT
2022-11-04 21:19:15,GNO,107.605877,2022-11-04 21:19:17.961,1.915270e+06,1972.0,1.877516e+05,1.736025e+05,3.461206e+04,124.500000,1.547535e+04,1.550025e+04,0.160643,83.376733,1.877516e+05,GNO,GNO
2022-11-04 21:19:30,MOB,104.611650,2022-11-04 21:19:35.414,3.752956e+06,8920.0,7.001551e+05,7.467351e+02,7.034197e+02,0.862000,6.441057e-01,6.456041e-01,0.232095,51.493475,7.001551e+05,MOB,MOB
2022-11-04 21:20:00,CVX,107.651869,2022-11-04 21:19:56.894,8.650398e+06,16339.0,2.469192e+06,2.813751e+03,4.640670e+02,5.529000,2.967473e+01,2.970657e+01,0.107197,85.842199,2.469192e+06,CVX,CVX
2022-11-04 21:20:00,IQ,104.026846,2022-11-04 21:20:04.101,1.066938e+07,1221.0,2.152107e+05,2.011480e+00,1.718967e+00,0.004650,2.152950e-05,2.162250e-05,0.430108,53.920610,2.152107e+05,IQ,IQ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-04 21:27:15,ADA,107.516340,2022-11-04 21:27:11.630,7.077777e+08,430572.0,1.629093e+08,1.374416e+06,1.355034e+06,0.427700,8.368834e+00,8.376258e+00,0.088633,50.355065,1.629093e+08,ADA,ADA
2022-11-04 21:27:15,FIL,106.701031,2022-11-04 21:27:11.630,1.866074e+08,167353.0,7.258999e+07,4.878507e+05,3.733230e+05,6.210000,4.152095e+01,4.158417e+01,0.152021,56.649509,7.258999e+07,FIL,FIL
2022-11-04 21:27:15,APT,106.482331,2022-11-04 21:27:11.630,9.632065e+06,317480.0,1.173310e+08,1.254192e+05,3.379139e+05,7.659700,8.264046e+01,8.267191e+01,0.038045,27.068902,1.173310e+08,APT,APT


In [2]:
import pandas as pd

crypto_exchange_info = 'crypto_logs/crypto_exchange_info.txt'
exchange_info = pd.read_csv(crypto_exchange_info, header=0, index_col=0)
exchange_info

,symbol,baseAsset,baseAssetPrecision,quoteAsset,quotePrecision,quoteAssetPrecision,baseCommissionPrecision,quoteCommissionPrecision,allowTrailingStop,cancelReplaceAllowed,min_price,max_price,tick_size,step_size,multiplier_up,multiplier_down
0,ETHBTC,ETH,8,BTC,8,8,8,8,True,True,0.000001,922327.0,0.000001,0.00010,5.0,0.2
1,LTCBTC,LTC,8,BTC,8,8,8,8,True,True,0.000001,100000.0,0.000001,0.00100,5.0,0.2
2,BNBBTC,BNB,8,BTC,8,8,8,8,True,True,0.000001,100000.0,0.000001,0.00100,5.0,0.2
3,NEOBTC,NEO,8,BTC,8,8,8,8,True,True,0.000001,100000.0,0.000001,0.01000,5.0,0.2
4,QTUMETH,QTUM,8,ETH,8,8,8,8,True,True,0.000001,1000.0,0.000001,0.10000,5.0,0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1421,BTCPLN,BTC,8,PLN,8,8,8,8,True,True,1.000000,9999319.0,1.000000,0.00001,5.0,0.2
1422,ETHPLN,ETH,8,PLN,8,8,8,8,True,True,1.000000,999996.0,1.000000,0.00010,5.0,0.2
1423,BUSDPLN,BUSD,8,PLN,8,8,8,8,True,True,0.001000,1000.0,0.001000,1.00000,1.2,0.8
1424,APTEUR,APT,8,EUR,8,8,8,8,True,True,0.000100,1000.0,0.000100,0.01000,5.0,0.2


In [ ]:
from cryptocurrency.authentication import Cryptocurrency_authenticator
from cryptocurrency.conversion_table import get_conversion_table, get_new_tickers
from cryptocurrency.exchange import Cryptocurrency_exchange
from cryptocurrency.conversion import convert_price

authenticator = Cryptocurrency_authenticator(use_keys=False, testnet=False)
client = authenticator.spot_client
exchange = Cryptocurrency_exchange(client=client, directory='crypto_logs')
exchange_info = exchange.info
conversion_table = get_conversion_table(client=client, exchange_info=exchange_info)
#assets = get_new_tickers(conversion_table=conversion_table)

symbol = 'BTCUSDT'
#original_size = exchange_info[exchange_info['symbol'] == symbol]['lastPrice']
quantity = df[df['symbol'] == symbol]['close'].iat[-1]
asset = exchange_info[exchange_info['symbol'] == symbol]['baseAsset'].iat[-1]

usdt_price = convert_price(size=1, from_asset=asset, to_asset='USDT', 
                           exchange_info=exchange_info, conversion_table=conversion_table)
usdt_price

In [ ]:
df.columns

In [ ]:
df = df[['symbol', 'lastPrice', 'volume', 'bidPrice', 'bidQty', 'askPrice', 'askQty']]
df = df.rename(columns={'lastPrice': 'close'})
df = df.pivot_table(index=['date'], columns=['symbol'], 
                    values=['close', 'volume', 'bidPrice', 'bidQty', 'askPrice', 'askQty'], 
                    aggfunc={'close': ['first', 'max', 'min', 'last'], 
                             'volume': 'last', 
                             'bidPrice': ['first', 'max', 'min', 'last'], 
                             'bidQty': 'sum', 
                             'askPrice': ['first', 'max', 'min', 'last'], 
                             'askQty': 'sum'})
df['volume'] = df['volume'].fillna(method='pad').fillna(0).diff(1)
df['bidQty']['sum'] = df['bidQty']['sum'].fillna(method='pad').fillna(0).diff(1)
df['askQty']['sum'] = df['askQty']['sum'].fillna(method='pad').fillna(0).diff(1)
df.columns = pd.MultiIndex.from_tuples([('_'.join(col[:2]), col[2]) for col in df.columns.values])
df = df.rename(columns={'close_first': 'open', 
                        'close_max': 'high', 
                        'close_min': 'low', 
                        'close_last': 'close', 
                        'volume_last': 'volume', 
                        'bidPrice_first': 'bid_open', 
                        'bidPrice_max': 'bid_high', 
                        'bidPrice_min': 'bid_low', 
                        'bidPrice_last': 'bid_close', 
                        'bidQty_sum': 'bid_volume', 
                        'askPrice_first': 'ask_open', 
                        'askPrice_max': 'ask_high', 
                        'askPrice_min': 'ask_low', 
                        'askPrice_last': 'ask_close', 
                        'askQty_sum': 'ask_volume'}, 
               level=0)
df

In [ ]:
df.columns

In [ ]:
df2.columns.get_level_values(1).unique()

In [ ]:
def resample_from_raw(df):
    df = df[['symbol', 'lastPrice', 'volume', 'quoteVolume']]
    df = df.rename(columns={'lastPrice': 'close', 
                            'volume': 'rolling_base_volume', 
                            'quoteVolume': 'rolling_quote_volume'})
    df = df.pivot_table(index=['date'], columns=['symbol'], 
                        values=['close', 'rolling_base_volume', 
                                'rolling_quote_volume'], 
                        aggfunc={'close': ['first', 'max', 'min', 'last'], 
                                 'rolling_base_volume': 'max', 
                                 'rolling_quote_volume': 'max'})
    df.columns = pd.MultiIndex.from_tuples([('_'.join(col[:2]), col[2]) for col in df.columns.values], 
                                           names=(None, 'symbol'))
    df = df.rename(columns={'close_first': 'open', 
                            'close_max': 'high', 
                            'close_min': 'low', 
                            'close_last': 'close', 
                            'rolling_base_volume_max': 'rolling_base_volume', 
                            'rolling_quote_volume_max': 'rolling_quote_volume'}, 
                   level=0)
    df['rolling_base_volume'] = df['rolling_base_volume'].fillna(method='pad')
    df['rolling_base_volume'].iloc[0] = 0
    df['rolling_quote_volume'] = df['rolling_quote_volume'].fillna(method='pad')
    df['rolling_quote_volume'].iloc[0] = 0
    df = df.sort_index().iloc[1:]
    df.columns = df.columns.swaplevel(0, 1)
    return df

df2 = resample_from_raw(df)
df2

In [ ]:
df2[(df2['NEBLBUSD']['volume'].diff() < 0)]['NEBLBUSD']['volume']

In [ ]:
#df2.columns = df2.columns.swaplevel(0, 1)
(~((df2[df2['volume'] > 0]['volume']).isna())).any().any()

In [ ]:
df2[(df2['volume'] > 0)]

In [ ]:
#%rm -rf crypto_logs